Q1

In [45]:
!python -m wget https://www.andrew.cmu.edu/user/mfarag/static/shopping_trends.csv



Saved under shopping_trends (4).csv


In [46]:
import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder.master("local[*]").appName('SparkTest').getOrCreate()

output of the description statistics

In [47]:
df = spark.read.csv('shopping_trends.csv', header=True, sep=",")
df.show(1, vertical=True)
df.describe().show()

-RECORD 0-------------------------------
 Customer ID              | 1           
 Age                      | 55          
 Gender                   | Male        
 Item Purchased           | Blouse      
 Category                 | Clothing    
 Purchase Amount (USD)    | 53          
 Location                 | Kentucky    
 Size                     | L           
 Color                    | Gray        
 Season                   | Winter      
 Review Rating            | 3.1         
 Subscription Status      | Yes         
 Payment Method           | Credit Card 
 Shipping Type            | Express     
 Discount Applied         | Yes         
 Promo Code Used          | Yes         
 Previous Purchases       | 14          
 Preferred Payment Method | Venmo       
 Frequency of Purchases   | Fortnightly 
only showing top 1 row

+-------+------------------+-----------------+------+--------------+-----------+---------------------+--------+----+------+------+------------------+-------

In [48]:
df=df.withColumnRenamed('Customer ID', 'customer_id')\
       .withColumnRenamed('Item Purchased', 'item_purchased')\
       .withColumnRenamed('Purchase Amount (USD)', 'purchase_amount_usd')\
       .withColumnRenamed('Review Rating', 'review_rating')\
       .withColumnRenamed('Subscription Status', 'subscription_status')\
       .withColumnRenamed('Payment Method', 'payment_method')\
       .withColumnRenamed('Shipping Type', 'shipping_type')\
       .withColumnRenamed('Discount Applied', 'discount_applied')\
       .withColumnRenamed('Promo Code Used', 'promo_code_used')\
       .withColumnRenamed('Previous Purchases', 'previous_purchases')\
       .withColumnRenamed('Preferred Payment Method', 'preferred_payment_method')\
       .withColumnRenamed('Frequency of Purchases', 'frequency_of_purchases')
df.printSchema()


root
 |-- customer_id: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- item_purchased: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- purchase_amount_usd: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- review_rating: string (nullable = true)
 |-- subscription_status: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- shipping_type: string (nullable = true)
 |-- discount_applied: string (nullable = true)
 |-- promo_code_used: string (nullable = true)
 |-- previous_purchases: string (nullable = true)
 |-- preferred_payment_method: string (nullable = true)
 |-- frequency_of_purchases: string (nullable = true)



In [49]:
df.columns

['customer_id',
 'Age',
 'Gender',
 'item_purchased',
 'Category',
 'purchase_amount_usd',
 'Location',
 'Size',
 'Color',
 'Season',
 'review_rating',
 'subscription_status',
 'payment_method',
 'shipping_type',
 'discount_applied',
 'promo_code_used',
 'previous_purchases',
 'preferred_payment_method',
 'frequency_of_purchases']

In [50]:
df = df.collect()

Q2

In [51]:
from neo4j import GraphDatabase

class Neo4JConnection:
    def __init__(self, uri, user, password):
        """Initialize the Neo4j connection with URI, username, and password."""
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        """Close the Neo4j connection."""
        if self.driver:
            self.driver.close()

    def execute_query(self, query, parameters=None):
        """Execute a Cypher query and return the result."""
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result.data()

In [52]:
uri = "neo4j+ssc://60282b47.databases.neo4j.io"
user = "neo4j"
password = "9VUwNvDamRhFZlPfuFFp_DGJE9eDen_ARYViMfSUTho"

# Initialize Neo4j connection
neo4j_conn = Neo4JConnection(uri, user, password)
driver = GraphDatabase.driver(
    uri,auth=(user,password)
)

In [59]:
def create_nodes_and_relationships(df, neo4j_conn):
    try:
        for row in df:
            query_user= """
            MERGE (u:User{customer_id:$customer_id})
            SET u.age=$age, u.gender=$gender
            """
            neo4j_conn.execute_query(query_user, parameters={
                "customer_id":row['customer_id'],  
                "age":row["Age"],  
                "gender":row["Gender"]  
            })
    
            query_item= """
            MERGE (i:Item {item_name:$item_name})
            """
            neo4j_conn.execute_query(query_item, parameters={
                "item_name":row["item_purchased"] 
            })
    
            query_relations= """
            MATCH (u:User {customer_id: $customer_id}), (i:Item {item_name: $item_name})
            MERGE (u)-[r: PURCHASED]-> (i)
            SET r.location= $location, r.purchase_amount=$purchase_amount, 
                r.size= $size, r.color= $color, r.season= $season,
                r.review_rating= $review_rating, r.subscription_status= $subscription_status, 
                r.payment_method= $payment_method, r.shipping_type= $shipping_type,
                r.discount_applied= $discount_applied, r.promo_code_used= $promo_code_used,
                r.previous_purchases= $previous_purchases, r.preferred_payment_method= $preferred_payment_method, 
                r.frequency_of_purchases= $frequency_of_purchases
            """
            neo4j_conn.execute_query(query_relations, parameters= {
                "customer_id":row['customer_id'],  
                "item_name": row['item_purchased'],  
                "location": row['Location'],  
                "purchase_amount": row['purchase_amount_usd'],  
                "size": row['Size'],  
                "color": row['Color'],  
                "season":row['Season'],  
                "review_rating": row['review_rating'],  
                "subscription_status": row['subscription_status'], 
                "payment_method": row['payment_method'],  
                "shipping_type": row['shipping_type'], 
                "discount_applied": row['discount_applied'],  
                "promo_code_used": row['promo_code_used'],  
                "previous_purchases": row['previous_purchases'],  
                "preferred_payment_method": row['preferred_payment_method'],  
                "frequency_of_purchases": row['frequency_of_purchases']  
            })
    finally:
        # Close the connection when done
        neo4j_conn.close()


In [60]:
create_nodes_and_relationships(df, neo4j_conn)

C:\Users\25353\AppData\Local\Temp\ipykernel_36792\1646435487.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


a query that retrieves the stored records

In [99]:
def showgraph(connection):
    query="""MATCH (u:User)-[p:PURCHASED]-> (i:Item)
    RETURN u, p, i
    LIMIT 5
    """
    result = connection.execute_query(query)
    print(result)
showgraph(neo4j_conn)

[{'u': {'gender': 'Female', 'customer_id': '2874', 'age': '44'}, 'p': ({'gender': 'Female', 'customer_id': '2874', 'age': '44'}, 'PURCHASED', {'item_name': 'Sweater'}), 'i': {'item_name': 'Sweater'}}, {'u': {'gender': 'Male', 'customer_id': '2', 'age': '19'}, 'p': ({'gender': 'Male', 'customer_id': '2', 'age': '19'}, 'PURCHASED', {'item_name': 'Sweater'}), 'i': {'item_name': 'Sweater'}}, {'u': {'gender': 'Male', 'customer_id': '19', 'age': '52'}, 'p': ({'gender': 'Male', 'customer_id': '19', 'age': '52'}, 'PURCHASED', {'item_name': 'Sweater'}), 'i': {'item_name': 'Sweater'}}, {'u': {'gender': 'Male', 'customer_id': '61', 'age': '50'}, 'p': ({'gender': 'Male', 'customer_id': '61', 'age': '50'}, 'PURCHASED', {'item_name': 'Sweater'}), 'i': {'item_name': 'Sweater'}}, {'u': {'gender': 'Male', 'customer_id': '94', 'age': '37'}, 'p': ({'gender': 'Male', 'customer_id': '94', 'age': '37'}, 'PURCHASED', {'item_name': 'Sweater'}), 'i': {'item_name': 'Sweater'}}]


C:\Users\25353\AppData\Local\Temp\ipykernel_36792\1646435487.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


references: “Merge - Cypher Manual.” Neo4j Graph Data Platform, neo4j.com/docs/cypher-manual/current/clauses/merge/. Accessed 21 Sept. 2024. 
Parameters - Cypher Manual. (n.d.). Neo4j Graph Data Platform. https://neo4j.com/docs/cypher-manual/current/syntax/parameters/

Q3

In [102]:
def get_50old_percent(connection):
    query = """
    MATCH (u:User) 
    WITH count(u) AS total_users 
    MATCH (u:User)
    WHERE toInteger(u.age) >= 50
    WITH total_users, count(u) AS count50
    RETURN toFloat(count50)/total_users * 100 AS percent50
    """
    result = connection.execute_query(query)
    print(result)
get_50old_percent(neo4j_conn)

[{'percent50': 39.97435897435898}]


C:\Users\25353\AppData\Local\Temp\ipykernel_36792\1646435487.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


Q4

In [78]:
def get_popular_hawaii(connection):
    query="""
    MATCH (u:User)-[p:PURCHASED]-> (i:Item)
    WHERE p.location="Hawaii"
    RETURN i.item_name, count(p) AS itemandcount
    ORDER BY itemandcount DESC
    LIMIT 1 
    """
    result= connection.execute_query(query)
    print(result[0]["i.item_name"])
get_popular_hawaii(neo4j_conn)

Handbag


C:\Users\25353\AppData\Local\Temp\ipykernel_36792\1646435487.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


Q5

In [86]:
def get_popular_season(connection):
    query="""
    MATCH (u:User)-[p:PURCHASED]-> (i:Item)
    WITH p.season AS season, count(p) AS season_count
    ORDER BY season_count DESC
    LIMIT 1
    RETURN season, season_count
    """
    result= connection.execute_query(query)
    print(result[0]['season'])
get_popular_season(neo4j_conn)

Spring


C:\Users\25353\AppData\Local\Temp\ipykernel_36792\1646435487.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


In [104]:
def get_season_shipping_method_number(connection):
    query="""
    MATCH (u:User)-[p:PURCHASED]->(i:Item)
    WHERE p.season = "Spring"
    WITH p.shipping_type AS method, count(p) AS count
    ORDER BY count DESC
    LIMIT 1
    RETURN method, count
    """
    result= connection.execute_query(query)
    print(result[0]['method'])
get_season_shipping_method_number(neo4j_conn)

2-Day Shipping


C:\Users\25353\AppData\Local\Temp\ipykernel_36792\1646435487.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


Q6

In [94]:
def top_3items_state(connection,statename):
    query = """
    MATCH (u:User)-[p:PURCHASED]-> (i:Item)
    WHERE p.location= $state
    WITH i.item_name AS item, COUNT(p) AS count
    ORDER BY count DESC
    LIMIT 3
    RETURN item
    """
    result= connection.execute_query(query, parameters={"state":statename})
    top_3items= [dictation["item"] for dictation in result]
    return top_3items
state1= "Kentucky"
items= top_3items_state(neo4j_conn, state1)
print(items)

['Handbag', 'Sweater', 'Shirt']


C:\Users\25353\AppData\Local\Temp\ipykernel_36792\1646435487.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:
